In [30]:
import pandas as pd
import os
import platform
os_type = platform.system()
os_file_path =os.getcwd()
if os_type == "Darwin": #running on mac
    relative_file_path = "/Data/retail_and_warehouse_sale.csv"
else: #running on windows or linux
    relative_file_path = "\\Data\\retail_and_warehouse_sale.csv"

data_file_path = os_file_path + relative_file_path
print(data_file_path)

df = pd.read_csv(data_file_path)
df.head()


/Users/mark/Documents/IRO/HCIIV/Project repo/Data/retail_and_warehouse_sale.csv


,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2020,1,REPUBLIC NATIONAL DISTRIBUTING CO,100009,BOOTLEG RED - 750ML,WINE,0.00,0.0,2.0
1,2020,1,PWSWN INC,100024,MOMENT DE PLAISIR - 750ML,WINE,0.00,1.0,4.0
2,2020,1,RELIABLE CHURCHILL LLLP,1001,S SMITH ORGANIC PEAR CIDER - 18.7OZ,BEER,0.00,0.0,1.0
3,2020,1,LANTERNA DISTRIBUTORS INC,100145,SCHLINK HAUS KABINETT - 750ML,WINE,0.00,0.0,1.0
4,2020,1,DIONYSOS IMPORTS INC,100293,SANTORINI GAVALA WHITE - 750ML,WINE,0.82,0.0,0.0
